In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction

Here our objective is to find who and why people are churning from bank.
keeping this things in mind i try to get more as possible useful insight.

What  this notebook contains:

Though i have gone throuh all pipelines of Data Science, But i focuses more on EDA and getting usefull insights.

What is in this Notebook.

1.Collecting data.

2.Exploratory data analysis.

3.Data wrangling.

4.Model building.


# Hope you will enjoy it and  plz upvote it for encouragement.


In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
%matplotlib inline

#Machine Learning

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df = pd.read_csv('../input/churn-modeling-dataset/Churn_Modelling.csv')

In [ ]:
df.head(5)

At first glance i don't think  'RowNumber'	'CustomerId'	'Surname' will evenfull in opur way.So drop them first.

In [ ]:
df.drop(['RowNumber','CustomerId','Surname'], axis = 1, inplace = True)

In [ ]:
df.shape

In [ ]:
df.describe()

From the above description we are seeing no null values. 

In [ ]:
df.dtypes

So here we are mainly seeing 2 types of features.

1.Numerical(float and int):CreditScore, Age, Tenure,Balance,EstimatedSalary .

2.Categorical:Geography and Gender, NumOfProducts, HasCrCard, IsActiveMember, Exited, 

In [ ]:
#Let's have the corelation among the features
drop_elements = ['Geography', 'Gender']
new_df = df.drop(drop_elements, axis = 1)
colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(new_df.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

From the above diagram we are seeing that 'Age', 'Balance' are greatly connected with target variable. 

In [ ]:
#Let'us see the realationship between numercal features and the target features.
creditscore_yes = df[df.Exited== 1].CreditScore
creditscore_no = df[df.Exited== 0].CreditScore

plt.hist([creditscore_yes, creditscore_no], color = ['green', 'red'], label = ['Exited = 1', 'Exited = 0'] )
plt.legend()

from the above histogram we are seeing that Credit score ranging from 600-750 didn't exited from the bank.
And we seeing a Normal distribution of creditscore.

In [ ]:
g = sns.FacetGrid(df, col='Exited')
g.map(plt.hist, 'Age', bins=30)

Here we are seeing that

1.Age ranging from 30-43 are most likely to not exited from the bank.

2.People who's age is near 35 and 40 are most in number in not exiting from bank.

3.And on the other hand around 40-45 age's people are exited from the bank.

4.Here most noticable thing is the oldest men in the are ranging in 76-80 and none of them are exited from the bank.


In [ ]:
Balance_yes = df[df.Exited== 1].Balance
Balance_no = df[df.Exited== 0].Balance

plt.hist([Balance_yes, Balance_no], color = ['green', 'red'], label = ['Exited = 1', 'Exited = 0'] )
plt.legend()

In [ ]:
#Here we are seeing a lot's persomn having 0 balance,let's check it.
x = len(df[df.Balance == 0].Balance)
x

Here most interest things to notice is 3617 customer of this bank has 0 balance.

And out odf 3617, 3017 didn't exited from the bank


In [ ]:
sns.countplot(x = 'Gender',hue = 'Exited', data = df)

In [ ]:
len(df[df.Gender == 'Male'].Gender)

In [ ]:
len(df[df.Gender == 'Female'].Gender)

Here we are seeing that Male and Female customer in the bank is same but Male person are not likely to exited from the bank then Female.

In [ ]:
sns.countplot(x = 'Tenure', hue = 'Exited', data = df)

Here we are seeing not so much difference.People are engaged with bank for 1-9 months has same mentality or thought process wheather to exit or not.


In [ ]:
len(df[df.Tenure == 10].Tenure)

Yes people who are engaged with the bank for 10 monthes is less in number that's why we are seeing less height in histogram.

In [ ]:
sns.countplot(x ='NumOfProducts', hue = 'Exited', data = df )

From the above histogram we are noticing that:

1.Customers who has NumOfProducts = 2 are mostly likely to stay in the bank.

2.And customers are totally exited from bank having 4 products.


## Analyze by pivoting featuress

Let's analyze our feature correlations by pivoting features against each other. It also makes sense doing so only for features which are categorical type.


In [ ]:
cat_features = df[['Geography','Gender', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',]].columns


df[['Geography', "Exited"]].groupby(['Geography'], as_index=False).mean().sort_values(by="Exited", ascending=False)

In [ ]:
df[['Gender', "Exited"]].groupby(['Gender'], as_index=False).mean().sort_values(by="Exited", ascending=False)

In [ ]:
df[['NumOfProducts', "Exited"]].groupby(['NumOfProducts'], as_index=False).mean().sort_values(by="Exited", ascending=False)

In [ ]:
df[['HasCrCard', "Exited"]].groupby(['HasCrCard'], as_index=False).mean().sort_values(by="Exited", ascending=False)

In [ ]:
df[['IsActiveMember', "Exited"]].groupby(['IsActiveMember'], as_index=False).mean().sort_values(by="Exited", ascending=False)

In [ ]:


sns.countplot(x = 'Geography', data = df)

In [ ]:

sns.countplot(x = 'Geography',hue = 'Exited', data = df)

Let's play the Geography features.And Group the dataset by Geography.

In [ ]:
g = df.groupby('Geography')

In [ ]:
France = g.get_group('France')
Germany = g.get_group('Germany')
Spain = g.get_group('Spain')



Let's find out the ratio of exiting and non exiting person of each country.

In [ ]:
def ratio(m):
    x = len(m[m.Exited == 1].Exited)
    y = len(m[m.Exited == 0].Exited)
    print(x)
    print(y)
    ratio = x/y
    
    return ratio

In [ ]:


ratio(France)

In [ ]:
ratio(Germany)


In [ ]:
ratio(Spain)

From the above information we are clearly seeing that

1.people from Germany are most likly to exit from bank.

2.And Bank has most of the customers from France.

and let's try to find out the causes.

In [ ]:
sns.distplot(France.Age)

In [ ]:
sns.distplot(Germany.Age)

In [ ]:
sns.distplot(Spain.Age)

if we observe this three diagram we will see that top most high peak is ranging between 30-40 in the case
of Spain and france.

And in the case of Germany ther is high peak between 38-45.

And if we compare this result with the earlier we get from Age distribution of total data, That's reason
people from Germany are are more likly to exit Because their majority people are in the age rangr 40-50.

Let's more clear with help of some more visualization.

In [ ]:
g = sns.FacetGrid(France, col='Exited')
g.map(plt.hist, 'Age', bins=30)

In [ ]:
g = sns.FacetGrid(Germany, col='Exited')
g.map(plt.hist, 'Age', bins=30)

In [ ]:
g = sns.FacetGrid(Spain, col='Exited')
g.map(plt.hist, 'Age', bins=30)

In [ ]:
len(France[France.Age > 78].Age)

In [ ]:
len(Germany[Germany.Age > 78].Age)

In [ ]:
len(Spain[Spain.Age > 78].Age)


This is another interesting point. Most of the senior person in the company are from France.That's why their ratio is less then rest two.
Becouse we have found that none of the senior customer are not exit.

In [ ]:
# let's Check Balane features with respect to each country.
x = len(France[France.Balance == 0].Balance)

x

In [ ]:
x = len(Germany[Germany.Balance == 0].Balance)
y = len(Germany[Germany.Balance >100000].Balance)

print('Customer  having balance, 0 = ' ,x)
print('Customer  having balance greater then, 100000  = ' ,y)


In [ ]:
x = len(Spain[Spain.Balance == 0].Balance)
x

From those above figure We get some importent information.

1.Every customer who are from Germany has balance And arround 2000customers has balance greater then 100000.They are likly to exit from the bank most.
Which means, there is strong possibility that they are facing problem while playing with money.

2.ALmost two third of person who are from France are not interested in having balance.


In [ ]:
sns.countplot(x = 'Gender', data = France)

In [ ]:
sns.countplot(x = 'Gender', data = Germany)

In [ ]:

sns.countplot(x = 'Gender', data = Spain)

In [ ]:
## distribution and checking for outliers in numeric features

features = df[['Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']].columns
for i in features:
    sns.boxplot(x="Exited",y=i,data=df)
    plt.title(i+" by "+"Exited")
    plt.show()

Here we are not seeing too much outliers except the age column.And we cann't delete or replace them because those rows are actually eventfull in our way.

# Conclusion:

All the deep insights that i have gotten by analysis.

1. Credit score ranging from 600-750 are most likely not to exited from the bank.

2.Age ranging from 30-43 are most likely to not exited from the bank.

3.People who's age is near 35 and 40 are most in number in not exiting from bank.

4.And on the other hand around 40-45 age's people are exited from the bank.

5.Here most noticable thing is the oldest men in the are ranging in 76-80 and none of them are exited from the bank.

6.Customers who has NumOfProducts = 2 are mostly likely to stay in the bank.

7.And customers are totally exited from bank having 4 products.

8.people from Germany are most likly to exit from bank.

9.And Bank has most of the customers from France.

10.People from Germany are are more likly to exit Because their majority people are in the age rangr 40-50.

11.Most of the senior person in the company are from France.

12.Every customer who are from Germany has balance And arround 2000 customers has balance greater then 100000.They are likly to exit from the bank most.
Which means, there is strong possibility that they are facing problem while playing with money.

13.ALmost two third of person who are from France are not interested in having balance.









In [ ]:
#Mapping the Gender and Geography features.
df['Gender'] = df['Gender'].map( {'Female': 1, 'Male': 0} ).astype(int)
df['Geography'] = df['Geography'].map( {'France': 0, 'Germany': 1,'Spain': 2} ).astype(int)


In [ ]:
#Let's scale down the data.

cols_to_scale = ['Tenure', 'CreditScore', 'Balance','EstimatedSalary']
scaler = MinMaxScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [ ]:
df.head()

# Features importence:

Here we are seeing that some of the features are most dominating  in this dataset.

1.Geography.

2.Age.

3.Balance.

4.NumOfProducts.

So let's play woth this.First of all i will try to train the clasifier with all possible features and the only dominating features to check the difference.

Let's Make some cluster of person based on the information that we got.

In [ ]:
Old_person = df[df.Age>70] #Didn't exit
Zer0_Balance =  df[df.Balance == 0] # Didn't exit

With_balance =  df[(df.Balance >= 0)  & (df.Geography == 1)]
NumOfProducts_2 =  df[(df.NumOfProducts == 2)] # Didn't exit


In [ ]:
NumOfProducts_2

In [ ]:
new_df = df.copy()

In [ ]:
X = df.drop(['Exited'], axis=1)
y = df.Exited

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
len(X_test.columns)

In [ ]:



model = keras.Sequential([
    keras.layers.Dense(30, input_shape = (10,),activation = 'sigmoid'),
     keras.layers.Dense(16,activation = 'relu' ),
    
     keras.layers.Dense(1,activation = 'sigmoid' )
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
    
)

In [ ]:
model.fit(X_train, y_train, epochs = 20,batch_size=32)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
y_p = model.predict(X_test)

In [ ]:
y_pred = []
for output in y_p:
    if output>.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
    

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
 print(classification_report(y_test,y_pred))

In [ ]:
cm = tf.math.confusion_matrix(labels = y_test, predictions = y_pred)
sns.heatmap(cm,annot = True)

In [ ]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, y_train) * 100, 2)
acc_log

# Model Selection with Hyperparameter tuning:

## Parameter Grid:

In [ ]:
model_params = {
    'svm': {
        'model' : svm.SVC(gamma = 'auto'),
        'params' : {
            'C' : [1,10,20,12,15],
            'kernel' : ['linear', 'poly', 'rbf', 'sigmoid']
        }
    },
    'random-forest' : {
        'model' : RandomForestClassifier(),
        'params' : {
             "n_estimators": [5, 10, 15, 20, 25], 
             "max_depth": [3, 5, 7, 9, 11, 13],
         }
    },
    'logisticregression': {
        'model' : LogisticRegression(),
        'params' : {
            'C' : [1,6,10],
           
        }
    },
    'decision_tree' :{
        'model' :  DecisionTreeClassifier(),
        'params' : {
             'max_depth': [3, 5, 7, 9, 11, 13],
        }
    }
    
}

In [ ]:
model_scores = []

for model_name ,mp in model_params.items():
    clf = RandomizedSearchCV(mp['model'],mp['params'], cv = 5,return_train_score = False,n_iter = 2)#for the computational purpose set n_ter = 2
    clf.fit(X_train,y_train)
    
    model_scores.append({
        'model' : model_name,
        'best_score' : clf.best_score_,
        'best_params' : clf.best_params_
        
    })

In [ ]:
df = pd.DataFrame(model_scores)
df